# Area Summary

This is an exercise to see if performing individual summaries of areas in the home are useful to understand the overall home summary.

## Setup client

In [1]:
from secrets import get_secret
import hass_data
import hass_client
import openai
import model_client

MODEL_ID = "mistral-7b-inst.gguf"

secrets = get_secret("hostport")
client: hass_client.HomeAssistantClient = await hass_data.create_client(
    get_secret("hostport"), get_secret("hass_token")
)
openai = openai.OpenAI(base_url=get_secret("openai_url"), api_key=get_secret("openai_key"))
model = model_client.ModelClient(openai, MODEL_ID)

In [2]:
from hass_client.models import Area, Device, Entity, State
import hass_data

home: hass_data.Home = await hass_data.get_home(client)

In [3]:
d = home.devices_by_area('closet')
d[0]['id'], d[0]['name']

('548f06f0d7eb7b79de037861657e3e19', 'garden')

In [4]:
e = home.entities_by_device_id('548f06f0d7eb7b79de037861657e3e19')
e[0]['entity_id'], e[0]['name']

('binary_sensor.garden_security_status', None)

In [5]:

state = home.get_state('binary_sensor.garden_security_status')
state['state'], state.get('attributes')

('on',
 {'malware': 'safe',
  'network': 'safe',
  'securitySetting': 'safe',
  'systemCheck': 'safe',
  'update': 'outOfDate',
  'userInfo': 'safe',
  'attribution': 'Data provided by Synology',
  'device_class': 'safety',
  'friendly_name': 'garden Security status'})

## Model

In [6]:
response = model.complete("You are a helpful assistant", "What is the capital of France?")
response

' The capital of France is Paris.'

## Area Summary

Attempt to get the list of devices in an area, the states of some of their entities, and create a summary.

In [7]:
def entity_data_for_area(area_id: str) -> list[str]:
    entity_data = []
    for device in home.devices_by_area(area_id):
        entities = home.entities_by_device_id(device['id'])
        if not entities:
            continue
        #print(device['name'])
        for entity in entities:
            state = home.get_state(entity['entity_id'])
            attributes = state['attributes']
            friendly_name = attributes.get('friendly_name')
            entity_data.append(f"{friendly_name}: {state['state']}")
    return entity_data


In [8]:
PROMPT = f"""
You are an agent running in Home Assistant. Your job is to summarize the status of an area
which will be fed as input into other agents. The user will feed in details about
areas and devices in the home, and you will respond with a summary of the status of the area.

Your summaries are succint, and do not mention boring details or things that seem
very mundane or minor. A one sentence summary is best.


Here is an example of the input and output:

Area: Driveway
  - Tesla: Charging
  - Tesla Battery: 90%
  - Tesla Estimated range: 200 miles
  - Pedestrian Gate: No motion
  - Sprinkler: Off

Summary: The car is almost charged.
"""


In [9]:

for area in home.areas:
    entity_data = entity_data_for_area(area['area_id'])
    entity_text = "\n".join([f"  - {data}" for data in entity_data])
    message = f"""Area: {area['name']}
{entity_text}

Summary:
"""
    response = model.complete(PROMPT, message)

    print(f"Area: {area['name']}")
    print(f"Summary: {response}")


Area: Living Room
Summary:  Summary: The living room lights are on.
Area: Kitchen
Summary:  The kitchen is dark, with the vacuum docked and a partially open Nook.
Area: Bedroom
Summary:  The bedroom is dark and quiet.
Area: Backyard
Summary:  Summary: The backyard is at a comfortable temperature, but there are no motion sensors or other devices active.
Area: Frontyard
Summary:  The car is almost charged and ready to go. The frontyard has a streaming sound, and the flume sensor is monitoring water usage.
Area: Dining Room
Summary:  The dining room is at a comfortable temperature and has natural light, but the chandelier is off.
Area: Office
Summary:  Summary: The office blinds are open.
Area: Garage
Summary:  Summary: The litter boxes are docked and the waste drawers are at 31.3% and 84.6%.
Area: Patio
Summary:  Summary: Nothing on the patio
Area: Family Room
Summary:  The family room is currently being used.
Area: Loft
Summary:  The loft is paused and the media playback status is pause

## Overall summary


In [13]:
OVERALL_PROMPT = f"""
You are an agent running in Home Assistant that is the overall house summary
agent. Other agents have summarized invidivual areas, and you are responsible
for picking out the important details, ignoring the mundane. You write succinct,
1 sentence summaries of the entire house that can be displayed on a dashboard.
You don't need to comment on every area, and can omit any that are mundane.

For example:
Areas:
- Living Room: The lights are off.
- Kitchen: The kitchen lights are off.
- Bedroom: The bedroom light is on.
- Backyard: The backyard is at a comfortable temperature, but there are no motion sensors or other devices active.
- Frontyard:  The car is idle. The the flume sensor is monitoring water usage.
- Dining Room: The dining room is at a comfortable temperature, the chandelier is off.
- Office: The office blinds are closed.
- Family Room: The family room is currently being used.

Summary: The bedroom light is on.

The user will enter the summaries from the agents, and you should respond
with a 1 sentence succinct summary of the entire home.
"""

message = """
Areas:
- Living Room: The living room lights are on.
- Kitchen: The kitchen is dark, with the vacuum docked and a partially open Nook.
- Bedroom: The bedroom is dark and quiet.
- Backyard: The backyard is at a comfortable temperature, but there are no motion sensors or other devices active.
- Frontyard:  The car is almost charged and ready to go. The frontyard has a streaming sound, and the flume sensor is monitoring water usage.
- Dining Room: The dining room is at a comfortable temperature and has natural light, but the chandelier is off.
- Office: The office blinds are open.
- Garage: The litter boxes are docked and the waste drawers are at 31.3% and 84.6%.
- Patio:  Nothing on the patio
- Family Room: The family room is currently being used.
- Loft:  The loft is paused and the media playback status is paused. The Roborock Upstairs is docked and the Frekvens display mode is set to date. The loft volume is at 90%.
- Closet: The garden is secure and has normal status.
- Cabana: The relative humidity in the cabana is at 59%, and the temperature is 71.8 degrees Fahrenheit.
- Attic: The attic temperature is high.
- Front Door The front door is being used and the porch is dimmed.

Summary: 
"""

response = model.complete(OVERALL_PROMPT, message)

print(f"Summary: {response}")


Summary:  The living room, office, bedroom, kitchen, dining room, patio, and family room are in use or have power on.
